# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /home/ajana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ajana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ajana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ajana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable',engine)
X = df.message.values
Y = df.iloc[:,4:]
X[1:10]

array(['Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I would like to receive the messages, thank you',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
stop_words = stopwords.words("english")
        
# def tokenize(text):
#     detected_urls = re.findall(url_regex, text)
#     for url in detected_urls:
#         text = text.replace(url, "urlplaceholder")

#     tokens = word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()
#     # lemmatize andremove stop words
#     tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

#     return tokens

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [4]:
tokenize(X[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [5]:

def display_results(y_test, y_pred):
    #labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    #print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [7]:
new_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier(objective='binary:logistic')))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
%%time

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ajana/anaconda3/lib/python3.7/site-package

Accuracy: related                   0.794990
request                   0.880744
offer                     0.996004
aid_related               0.730905
medical_help              0.919471
medical_products          0.950669
search_and_rescue         0.971415
security                  0.981405
military                  0.968803
water                     0.946519
food                      0.912402
shelter                   0.925004
clothing                  0.984632
money                     0.975872
missing_people            0.987859
refugees                  0.968803
death                     0.960043
other_aid                 0.866298
infrastructure_related    0.932688
transport                 0.956355
buildings                 0.950976
electricity               0.982480
tools                     0.994621
hospitals                 0.987859
shops                     0.996465
aid_centers               0.988167
other_infrastructure      0.953896
weather_related           0.836176
floods    

In [10]:
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4942
               request       0.81      0.39      0.53      1109
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.53      0.62      2717
          medical_help       0.49      0.07      0.13       522
      medical_products       0.72      0.08      0.15       338
     search_and_rescue       1.00      0.04      0.07       193
              security       0.50      0.01      0.02       121
              military       0.80      0.07      0.14       215
                 water       0.81      0.19      0.31       408
                  food       0.85      0.27      0.41       732
               shelter       0.82      0.21      0.34       584
              clothing       0.67      0.02      0.04       101
                 money       0.71      0.03      0.06       160
        missing_people       0.14      

/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


From the results, two things can be inferred, there is something wrong with **related** column and **child_alone** column.

In [11]:
# investigate "related" and "child-alone" column
Y["related"].value_counts()

1    19906
0     6122
Name: related, dtype: int64

In [12]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

Few importatnt words like **water, blocked road, medical supplies** are used during a disaster response. So we can create a Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** and, **LengthExtractor** and add them to our pipeline.


In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [14]:
# https://www.guru99.com/pos-tagging-chunking-nltk.html

class StartingNounExtractor(BaseEstimator, TransformerMixin):

    def starting_noun(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['NN', 'NNS', 'NNP', 'NNPS'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_noun)
        return pd.DataFrame(X_tagged)

In [15]:
class LengthExtractor(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return  pd.Series(X).apply(len).values.reshape(-1,1)

### Using FeatureUnion

In [25]:
test_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('length', LengthExtractor()),
            ('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [27]:
boost_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('length', LengthExtractor()),
            ('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(XGBClassifier(objective='binary:logistic')))
    ])

In [26]:
%%time

# train classifier
test_pipeline.fit(X_train, y_train)

#predict on test data
y_pred = test_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred)

Accuracy: related                   0.798832
request                   0.895190
offer                     0.996004
aid_related               0.772092
medical_help              0.919471
medical_products          0.948824
search_and_rescue         0.971569
security                  0.981251
military                  0.967727
water                     0.946212
food                      0.926080
shelter                   0.929461
clothing                  0.984939
money                     0.975872
missing_people            0.988013
refugees                  0.969110
death                     0.958506
other_aid                 0.867988
infrastructure_related    0.932842
transport                 0.956816
buildings                 0.951591
electricity               0.982480
tools                     0.994621
hospitals                 0.988013
shops                     0.996465
aid_centers               0.988167
other_infrastructure      0.954664
weather_related           0.865683
floods    

In [28]:
%%time

# train classifier
boost_pipeline.fit(X_train, y_train)

#predict on test data
y_pred = boost_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred)

Accuracy: related                   0.819579
request                   0.902413
offer                     0.995851
aid_related               0.766866
medical_help              0.924696
medical_products          0.956355
search_and_rescue         0.972952
security                  0.981405
military                  0.970340
water                     0.962195
food                      0.951591
shelter                   0.948978
clothing                  0.989550
money                     0.978638
missing_people            0.989703
refugees                  0.970032
death                     0.968649
other_aid                 0.869986
infrastructure_related    0.932688
transport                 0.959428
buildings                 0.958353
electricity               0.983249
tools                     0.994160
hospitals                 0.988474
shops                     0.996312
aid_centers               0.987859
other_infrastructure      0.952205
weather_related           0.881820
floods    

In [22]:
length_pipeline = Pipeline([        
            ('length', LengthExtractor()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [24]:
%%time

# train classifier
length_pipeline.fit(X_train, y_train)

#predict on test data
y_pred = length_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred)

Accuracy: related                   0.756416
request                   0.827878
offer                     0.996004
aid_related               0.592900
medical_help              0.919010
medical_products          0.946980
search_and_rescue         0.970186
security                  0.980944
military                  0.966344
water                     0.935915
food                      0.885815
shelter                   0.910250
clothing                  0.984478
money                     0.974182
missing_people            0.988167
refugees                  0.969264
death                     0.954664
other_aid                 0.864915
infrastructure_related    0.930075
transport                 0.954203
buildings                 0.945904
electricity               0.981712
tools                     0.994160
hospitals                 0.988013
shops                     0.996312
aid_centers               0.987859
other_infrastructure      0.953742
weather_related           0.710927
floods    

As we can see adding Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** and, **LengthExtractor** to our pipeline, improves the accuracy. Also, XGBoost classifier workes better than random forest. So we'll apply GridsearchCV on XGBoost.


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.